In [22]:
from beir.datasets.data_loader import GenericDataLoader

In [23]:
from beir import util
dataset = "arguana"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
data_path = f"/dss/dsshome1/07/ra65bex2/srawat/{dataset}"

# Download and unzip the dataset
data_path = util.download_and_unzip(url, data_path)

In [24]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

  0%|          | 0/8674 [00:00<?, ?it/s]

In [25]:
contrastive_pairs=[]
import random
c=0
for query_id, relevant_docs in qrels.items():
    try:
        query_text = queries[query_id]
        for doc_id in relevant_docs:
            positive = corpus[doc_id]["text"]
        #print(relevant_docs)
        positive_doc_ids = set(relevant_docs)
        all_doc_ids = set(corpus.keys())
        negative_doc_ids = all_doc_ids - positive_doc_ids
        negative_doc_ids=list(negative_doc_ids)
        negative_doc_samples = random.sample(negative_doc_ids, k=5)
        negatives=[]
        for neg_doc_id in negative_doc_samples:
            negative_doc_text = corpus[neg_doc_id]["text"]
            negatives.append(negative_doc_text)
        contrastive_pairs.append({
            "anchor": query_text,
            "positive": positive,
            "negatives": negatives
        })
    except:
        c=c+1

In [26]:
c

5

In [27]:
len(contrastive_pairs)

1401

In [28]:
len(qrels)

1406

In [29]:
from torch.utils.data import DataLoader

In [30]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [31]:
contrastive_dataset = ContrastiveDataset(contrastive_pairs)

In [32]:
data_loader = DataLoader(contrastive_dataset, batch_size=32, shuffle=True)

In [33]:
len(data_loader)

44

In [34]:
import torch
file_path="/dss/dsshome1/07/ra65bex2/srawat/contrastive_learning/v1.1/app_average/average_checkpoint_epoch_3.pth"
lora_model = torch.load(file_path)

/tmp/ipykernel_317602/2192687965.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_model = torch.load(file_path)


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lora_model = lora_model.to(device)

In [36]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [37]:
def cosine_distance(x, y):
    return 1 - torch.nn.functional.cosine_similarity(x, y, dim=-1)

In [38]:
def avg_embedding(inputs, model):
    input=model(**inputs)
    input_last_hidden_state=input.last_hidden_state
    input_attention_mask = inputs['attention_mask']
    input_masked_embeddings = input_last_hidden_state * input_attention_mask.unsqueeze(-1)
    input_sum_embeddings = torch.sum(input_masked_embeddings, dim=1)
    input_token_counts = torch.sum(input_attention_mask, dim=1).unsqueeze(-1)
    input_avg_embeddings = input_sum_embeddings / input_token_counts
    return(input_avg_embeddings)

In [39]:
def evaluate_mrr(model, data_loader_val, distance_fn):
    model.eval()  

    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = avg_embedding(anchor_input, model)
            positive_embedding = avg_embedding(positive_input, model)
            negative_embedding = [avg_embedding(tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device), model) for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
            
            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr

In [40]:
mrr_validation = evaluate_mrr(lora_model, data_loader, cosine_distance)
print(mrr_validation)

0.6639305430595404
